In [1]:
import torch
from datasets import list_datasets, load_dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments

from sklearn.metrics import accuracy_score, f1_score


# DATA
# dataset = load_dataset('emotion')
dataset = load_dataset('tamilmixsentiment')
dataset


# PARAMS
num_labels = len(set(dataset['train']['label']))
print('num_labels:', num_labels)
model_ckpt = 'distilbert-base-uncased'


# DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


# TOKENS
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)


dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)
dataset_encoded


print(dataset_encoded['train'].format)
dataset_encoded.set_format('torch')
print(dataset_encoded['train'].format)


# MODEL
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=num_labels
).to(device)

print('model.device', model.device)


def compute_metrics(pred):
    res = {'acc': 1.0, 'F1': 2.0}
    return res


training_args = TrainingArguments(
    output_dir='resultados',
    num_train_epochs=2,
    learning_rate=2e-5,
    disable_tqdm=False,
    evaluation_strategy='steps',
    logging_steps=1,
    report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded['train'],
    eval_dataset=dataset_encoded['validation'],
    tokenizer=tokenizer,
)

print('trainer.args.device:', trainer.args.device)

Found cached dataset tamilmixsentiment (/home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390)


  0%|          | 0/3 [00:00<?, ?it/s]

num_labels: 5


Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390/cache-f7dffd34dfe35a97.arrow
Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390/cache-d2f3a02e4585e68f.arrow
Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390/cache-29874ec5146d3c07.arrow


{'type': None, 'format_kwargs': {}, 'columns': ['text', 'label', 'input_ids', 'attention_mask'], 'output_all_columns': False}
{'type': 'torch', 'format_kwargs': {}, 'columns': ['text', 'label', 'input_ids', 'attention_mask'], 'output_all_columns': False}


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

model.device cpu
trainer.args.device: cpu


In [2]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11335
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2834
  Number of trainable parameters = 66957317
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the

Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1260
  Batch size = 8


KeyboardInterrupt: 